In [1]:
from __future__ import absolute_import
from __future__ import print_function
import xml.etree.cElementTree as ET
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
from scipy.interpolate import make_interp_spline
import os
import sys
import math
import fnmatch
import warnings
from glob import glob
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import sumolib

In [3]:
NetName = "Tainan.net.xml"
net = sumolib.net.readNet(NetName)
edges = net.getEdges()
Lane_dict = {}

for edge in edges:
    lanes = edge.getLanes()
    for lane in lanes:
        Lane_dict[lane.getID()] = lane.getLength()
#print(len(Lane_dict.keys()))
lane_list = ["-496253687#1_0","250710099#0_0","-24452786#6_0","-160253722#7_0","-24452786#2_0","71285598#5_0","108037210#0_0","-496332201#4_0","-286094870#6_0","-227913938#10_0","307096544#4_0"]

In [4]:
# 45 30 (30,45)
warnings.filterwarnings("ignore")
for i in range(45):
    for lane in lane_list:
    #for lane in Lane_dict.keys():
            esti_data = pd.DataFrame(columns=["time","vehid","Pre_ins","Bd","avgspeed"])
            try:
                with open("esti_0/out"+str(i)+"/"+lane+".txt",'r') as rf:
                    for line in rf:
                        line.replace("\n", "")
                        if line != "":
                            token = line.split(",")
                            time = float(token[0])
                            veh_id = float(token[1])

                            Pre_ins = float(token[3].replace("[[","").replace("]]",""))
                            Bd = float(token[4])
                            avgspd = float(token[5].replace("[[","").replace("]]",""))
                            newrow = {"time":time,"vehid":veh_id,"Pre_ins":Pre_ins,"Bd":Bd,"avgspeed":avgspd}
                            esti_data = esti_data._append(newrow, ignore_index=True)
                    esti_data["vehid"] = esti_data["vehid"].astype(int)
                    esti_data = esti_data.sort_values(by=["time"])
                    esti_data.to_csv("esti_0/out"+str(i)+"/"+lane+".csv" , index=False)
            except Exception as e:
                #print(e)
                continue
    

    

In [5]:
for i in range(45): 
    warnings.filterwarnings("ignore")
    try: 
        os.system("del esti_0\out" +str(i)+"\*.txt")
    except Exception as e:
        print(e)
        continue

In [6]:
# Calculate Avg of RMSE based on road_seg
warnings.filterwarnings("ignore")
for lane in lane_list:
#for lane in Lane_dict.keys():
    counter = 0
    rmse_sum = 0.0
    for j in range(45):
        r_file_path = "real_0/out"+str(j)+"/"+lane+"_finalavg.csv"
        r = os.path.isfile(r_file_path)
        e_file_path = "esti_0/out"+str(j)+"/"+lane+".csv"
        e = os.path.isfile(e_file_path)
        if r == True and e == True:
            real_data = pd.read_csv("real_0/out"+str(j)+"/"+lane+"_finalavg.csv")    
            esti_data = pd.read_csv("esti_0/out"+str(j)+"/"+lane+".csv")
        else:
            continue
        try:
            RealData_dict = {}
            RealDataTime_dict = {}
            RealDataID_list = real_data["vehid"].tolist()
            RealDataAvg_list = real_data["avgspeed"].tolist()
            entry_list = real_data["entry"].tolist()
            InstData_dict = {}
            inst_list = real_data["instspeed"].tolist()

            for i in range(len(RealDataID_list)):
                ID = RealDataID_list[i]
                avg = RealDataAvg_list[i]
                entry = entry_list[i]
                inst = inst_list[i]
                RealData_dict[ID] = avg
                RealDataTime_dict[ID] = entry
                InstData_dict[ID] = inst

            EstiData_dict = {}
            EstiDataID_list = esti_data["vehid"].tolist()
            EstiDataAvg_list = esti_data["avgspeed"].tolist()
            
            esti_plot = []
            real_plot = []
            inst_plot = []
            time_line = []
            for i in range(len(EstiDataID_list)):
                ID = EstiDataID_list[i]                
                avg = EstiDataAvg_list[i]                
                if ID in RealData_dict:
                    if RealData_dict[ID] < 2:
                        continue
                    EstiData_dict[ID] = avg
                    esti_plot.append(avg)
                    real_plot.append(RealData_dict[ID])
                    inst_plot.append(InstData_dict[ID])
                    time_line.append(RealDataTime_dict[ID])
            total = 0
            for i in range(len(esti_plot)):
                total += (esti_plot[i] - real_plot[i]) * (esti_plot[i] - real_plot[i])
        except Exception as e:
            #print(e)
            continue
        
        if j==0 and len(esti_plot) > 0:            
            with open("Result/Roads/"+lane+"_AF3_30_0.csv", "w", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Day","RMSE"])
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        elif j>0 and len(esti_plot) > 0:
            with open("Result/Roads/"+lane+"_AF3_30_0.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        else:
            continue
    if counter > 0:
        Avg_RMSE = rmse_sum/counter    
        #print(str(counter),Avg_RMSE)
        with open("Result/Roads/"+lane+"_AF3_30_0.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Average =",Avg_RMSE])

In [7]:
# 45 30 (30,45)
warnings.filterwarnings("ignore")
for i in range(45):
    for lane in lane_list:
    #for lane in Lane_dict.keys():
            esti_data = pd.DataFrame(columns=["time","vehid","Pre_ins","Bd","avgspeed"])
            try:
                with open("esti_1/out"+str(i)+"/"+lane+".txt",'r') as rf:
                    for line in rf:
                        line.replace("\n", "")
                        if line != "":
                            token = line.split(",")
                            time = float(token[0])
                            veh_id = float(token[1])

                            Pre_ins = float(token[3].replace("[[","").replace("]]",""))
                            Bd = float(token[4])
                            avgspd = float(token[5].replace("[[","").replace("]]",""))
                            newrow = {"time":time,"vehid":veh_id,"Pre_ins":Pre_ins,"Bd":Bd,"avgspeed":avgspd}
                            esti_data = esti_data._append(newrow, ignore_index=True)
                    esti_data["vehid"] = esti_data["vehid"].astype(int)
                    esti_data = esti_data.sort_values(by=["time"])
                    esti_data.to_csv("esti_1/out"+str(i)+"/"+lane+".csv" , index=False)
            except Exception as e:
                #print(e)
                continue
    

    

In [8]:
for i in range(45): 
    warnings.filterwarnings("ignore")
    try: 
        os.system("del esti_1\out" +str(i)+"\*.txt")
    except Exception as e:
        print(e)
        continue

In [9]:
# Calculate Avg of RMSE based on road_seg
warnings.filterwarnings("ignore")
for lane in lane_list:
#for lane in Lane_dict.keys():
    counter = 0
    rmse_sum = 0.0
    for j in range(45):
        r_file_path = "real_1/out"+str(j)+"/"+lane+"_finalavg.csv"
        r = os.path.isfile(r_file_path)
        e_file_path = "esti_1/out"+str(j)+"/"+lane+".csv"
        e = os.path.isfile(e_file_path)
        if r == True and e == True:
            real_data = pd.read_csv("real_1/out"+str(j)+"/"+lane+"_finalavg.csv")    
            esti_data = pd.read_csv("esti_1/out"+str(j)+"/"+lane+".csv")
        else:
            continue
        try:
            RealData_dict = {}
            RealDataTime_dict = {}
            RealDataID_list = real_data["vehid"].tolist()
            RealDataAvg_list = real_data["avgspeed"].tolist()
            entry_list = real_data["entry"].tolist()
            InstData_dict = {}
            inst_list = real_data["instspeed"].tolist()

            for i in range(len(RealDataID_list)):
                ID = RealDataID_list[i]
                avg = RealDataAvg_list[i]
                entry = entry_list[i]
                inst = inst_list[i]
                RealData_dict[ID] = avg
                RealDataTime_dict[ID] = entry
                InstData_dict[ID] = inst

            EstiData_dict = {}
            EstiDataID_list = esti_data["vehid"].tolist()
            EstiDataAvg_list = esti_data["avgspeed"].tolist()
            
            esti_plot = []
            real_plot = []
            inst_plot = []
            time_line = []
            for i in range(len(EstiDataID_list)):
                ID = EstiDataID_list[i]                
                avg = EstiDataAvg_list[i]                
                if ID in RealData_dict:
                    if RealData_dict[ID] < 2:
                        continue
                    EstiData_dict[ID] = avg
                    esti_plot.append(avg)
                    real_plot.append(RealData_dict[ID])
                    inst_plot.append(InstData_dict[ID])
                    time_line.append(RealDataTime_dict[ID])
            total = 0
            for i in range(len(esti_plot)):
                total += (esti_plot[i] - real_plot[i]) * (esti_plot[i] - real_plot[i])
        except Exception as e:
            #print(e)
            continue
        
        if j==0 and len(esti_plot) > 0:            
            with open("Result/Roads/"+lane+"_AF3_30_1.csv", "w", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Day","RMSE"])
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        elif j>0 and len(esti_plot) > 0:
            with open("Result/Roads/"+lane+"_AF3_30_1.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        else:
            continue
    if counter > 0:
        Avg_RMSE = rmse_sum/counter    
        #print(str(counter),Avg_RMSE)
        with open("Result/Roads/"+lane+"_AF3_30_1.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Average =",Avg_RMSE])

In [10]:
# 45 30 (30,45)
warnings.filterwarnings("ignore")
for i in range(45):
    for lane in lane_list:
    #for lane in Lane_dict.keys():
            esti_data = pd.DataFrame(columns=["time","vehid","Pre_ins","Bd","avgspeed"])
            try:
                with open("esti_2/out"+str(i)+"/"+lane+".txt",'r') as rf:
                    for line in rf:
                        line.replace("\n", "")
                        if line != "":
                            token = line.split(",")
                            time = float(token[0])
                            veh_id = float(token[1])

                            Pre_ins = float(token[3].replace("[[","").replace("]]",""))
                            Bd = float(token[4])
                            avgspd = float(token[5].replace("[[","").replace("]]",""))
                            newrow = {"time":time,"vehid":veh_id,"Pre_ins":Pre_ins,"Bd":Bd,"avgspeed":avgspd}
                            esti_data = esti_data._append(newrow, ignore_index=True)
                    esti_data["vehid"] = esti_data["vehid"].astype(int)
                    esti_data = esti_data.sort_values(by=["time"])
                    esti_data.to_csv("esti_2/out"+str(i)+"/"+lane+".csv" , index=False)
            except Exception as e:
                #print(e)
                continue
    

    

In [11]:
for i in range(45): 
    warnings.filterwarnings("ignore")
    try: 
        os.system("del esti_2\out" +str(i)+"\*.txt")
    except Exception as e:
        print(e)
        continue

In [12]:
# Calculate Avg of RMSE based on road_seg
warnings.filterwarnings("ignore")
for lane in lane_list:
#for lane in Lane_dict.keys():
    counter = 0
    rmse_sum = 0.0
    for j in range(45):
        r_file_path = "real_2/out"+str(j)+"/"+lane+"_finalavg.csv"
        r = os.path.isfile(r_file_path)
        e_file_path = "esti_2/out"+str(j)+"/"+lane+".csv"
        e = os.path.isfile(e_file_path)
        if r == True and e == True:
            real_data = pd.read_csv("real_2/out"+str(j)+"/"+lane+"_finalavg.csv")    
            esti_data = pd.read_csv("esti_2/out"+str(j)+"/"+lane+".csv")
        else:
            continue
        try:
            RealData_dict = {}
            RealDataTime_dict = {}
            RealDataID_list = real_data["vehid"].tolist()
            RealDataAvg_list = real_data["avgspeed"].tolist()
            entry_list = real_data["entry"].tolist()
            InstData_dict = {}
            inst_list = real_data["instspeed"].tolist()

            for i in range(len(RealDataID_list)):
                ID = RealDataID_list[i]
                avg = RealDataAvg_list[i]
                entry = entry_list[i]
                inst = inst_list[i]
                RealData_dict[ID] = avg
                RealDataTime_dict[ID] = entry
                InstData_dict[ID] = inst

            EstiData_dict = {}
            EstiDataID_list = esti_data["vehid"].tolist()
            EstiDataAvg_list = esti_data["avgspeed"].tolist()
            
            esti_plot = []
            real_plot = []
            inst_plot = []
            time_line = []
            for i in range(len(EstiDataID_list)):
                ID = EstiDataID_list[i]                
                avg = EstiDataAvg_list[i]                
                if ID in RealData_dict:
                    if RealData_dict[ID] < 2:
                        continue
                    EstiData_dict[ID] = avg
                    esti_plot.append(avg)
                    real_plot.append(RealData_dict[ID])
                    inst_plot.append(InstData_dict[ID])
                    time_line.append(RealDataTime_dict[ID])
            total = 0
            for i in range(len(esti_plot)):
                total += (esti_plot[i] - real_plot[i]) * (esti_plot[i] - real_plot[i])
        except Exception as e:
            #print(e)
            continue
        
        if j==0 and len(esti_plot) > 0:            
            with open("Result/Roads/"+lane+"_AF3_30_2.csv", "w", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Day","RMSE"])
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        elif j>0 and len(esti_plot) > 0:
            with open("Result/Roads/"+lane+"_AF3_30_2.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        else:
            continue
    if counter > 0:
        Avg_RMSE = rmse_sum/counter    
        #print(str(counter),Avg_RMSE)
        with open("Result/Roads/"+lane+"_AF3_30_2.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Average =",Avg_RMSE])

In [13]:
# 45 30 (30,45)
warnings.filterwarnings("ignore")
for i in range(45):
    for lane in lane_list:
    #for lane in Lane_dict.keys():
            esti_data = pd.DataFrame(columns=["time","vehid","Pre_ins","Bd","avgspeed"])
            try:
                with open("esti_3/out"+str(i)+"/"+lane+".txt",'r') as rf:
                    for line in rf:
                        line.replace("\n", "")
                        if line != "":
                            token = line.split(",")
                            time = float(token[0])
                            veh_id = float(token[1])

                            Pre_ins = float(token[3].replace("[[","").replace("]]",""))
                            Bd = float(token[4])
                            avgspd = float(token[5].replace("[[","").replace("]]",""))
                            newrow = {"time":time,"vehid":veh_id,"Pre_ins":Pre_ins,"Bd":Bd,"avgspeed":avgspd}
                            esti_data = esti_data._append(newrow, ignore_index=True)
                    esti_data["vehid"] = esti_data["vehid"].astype(int)
                    esti_data = esti_data.sort_values(by=["time"])
                    esti_data.to_csv("esti_3/out"+str(i)+"/"+lane+".csv" , index=False)
            except Exception as e:
                #print(e)
                continue
    

    

In [14]:
for i in range(45): 
    warnings.filterwarnings("ignore")
    try: 
#         # File name
#         file = '*.txt'
  
#         # File location
#         location = "real/out"+str(i)
  
#         # Path
#         path = os.path.join(location, file)
#         os.remove(path)
        os.system("del esti_3\out" +str(i)+"\*.txt")
    except Exception as e:
        print(e)
        continue

In [15]:
# Calculate Avg of RMSE based on road_seg
warnings.filterwarnings("ignore")
for lane in lane_list:
#for lane in Lane_dict.keys():
    counter = 0
    rmse_sum = 0.0
    for j in range(45):
        r_file_path = "real_3/out"+str(j)+"/"+lane+"_finalavg.csv"
        r = os.path.isfile(r_file_path)
        e_file_path = "esti_3/out"+str(j)+"/"+lane+".csv"
        e = os.path.isfile(e_file_path)
        if r == True and e == True:
            real_data = pd.read_csv("real_3/out"+str(j)+"/"+lane+"_finalavg.csv")    
            esti_data = pd.read_csv("esti_3/out"+str(j)+"/"+lane+".csv")
        else:
            continue
        try:
            RealData_dict = {}
            RealDataTime_dict = {}
            RealDataID_list = real_data["vehid"].tolist()
            RealDataAvg_list = real_data["avgspeed"].tolist()
            entry_list = real_data["entry"].tolist()
            InstData_dict = {}
            inst_list = real_data["instspeed"].tolist()

            for i in range(len(RealDataID_list)):
                ID = RealDataID_list[i]
                avg = RealDataAvg_list[i]
                entry = entry_list[i]
                inst = inst_list[i]
                RealData_dict[ID] = avg
                RealDataTime_dict[ID] = entry
                InstData_dict[ID] = inst

            EstiData_dict = {}
            EstiDataID_list = esti_data["vehid"].tolist()
            EstiDataAvg_list = esti_data["avgspeed"].tolist()
            
            esti_plot = []
            real_plot = []
            inst_plot = []
            time_line = []
            for i in range(len(EstiDataID_list)):
                ID = EstiDataID_list[i]                
                avg = EstiDataAvg_list[i]                
                if ID in RealData_dict:
                    if RealData_dict[ID] < 2:
                        continue
                    EstiData_dict[ID] = avg
                    esti_plot.append(avg)
                    real_plot.append(RealData_dict[ID])
                    inst_plot.append(InstData_dict[ID])
                    time_line.append(RealDataTime_dict[ID])
            total = 0
            for i in range(len(esti_plot)):
                total += (esti_plot[i] - real_plot[i]) * (esti_plot[i] - real_plot[i])
        except Exception as e:
            #print(e)
            continue
        
        if j==0 and len(esti_plot) > 0:            
            with open("Result/Roads/"+lane+"_AF3_30_3.csv", "w", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Day","RMSE"])
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        elif j>0 and len(esti_plot) > 0:
            with open("Result/Roads/"+lane+"_AF3_30_3.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                RMSE = math.sqrt(total/len(esti_plot))
                rmse_sum+=RMSE
                counter+=1
                d.writerow([str(j),RMSE])
        else:
            continue
    if counter > 0:
        Avg_RMSE = rmse_sum/counter    
        #print(str(counter),Avg_RMSE)
        with open("Result/Roads/"+lane+"_AF3_30_3.csv", "a", newline='') as fp:    
                d = csv.writer(fp)
                d.writerow(["Average =",Avg_RMSE])